In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(children=[html.H1('SNHU CS-340 Dashboard - Andrew Black 2022-08-08'),
                                 html.Img(src='https://i.imgur.com/dTImo2h.png', style={'max-width': '100px'})])),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        dcc.RadioItems(
            options=[{'label': x, 'value': x} for x in ['Water Rescue', 'Mountain Rescue', 'Disaster Rescue', 'Reset']],
            value='Reset',
            id='datatable-filter',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns[1:]
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        style={'display': 'flex'},
        children=[
            dcc.Graph(id='pie-chart'),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ]
    )
])


#############################################
# Interaction Between Components / Controller
#############################################
# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('datatable-id', 'data'),
    [Input('datatable-filter', 'value')]
)
def filter_graph(v):
    if v == 'Water Rescue':
        filtered_df = pd.DataFrame.from_records(shelter.read({
            'breed': {
                '$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
            },
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }))
    elif v == 'Mountain Rescue':
        filtered_df = pd.DataFrame.from_records(shelter.read({
            'breed': {
                '$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
            },
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }))
    elif v == 'Disaster Rescue':
        filtered_df = pd.DataFrame.from_records(shelter.read({
            'breed': {
                '$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
            },
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }))
    else:
        filtered_df = df
    return filtered_df.to_dict('records')


@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-filter', 'value')]
)
def filter_pie_chart(v):
    if v == 'Water Rescue':
        filtered_df = pd.DataFrame.from_records(shelter.read({
            'breed': {
                '$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
            },
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }))
    elif v == 'Mountain Rescue':
        filtered_df = pd.DataFrame.from_records(shelter.read({
            'breed': {
                '$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
            },
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }))
    elif v == 'Disaster Rescue':
        filtered_df = pd.DataFrame.from_records(shelter.read({
            'breed': {
                '$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
            },
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }))
    else:
        filtered_df = df
        return px.pie(filtered_df, names=filtered_df['breed'].value_counts().nlargest(5).keys().tolist(), values=filtered_df['breed'].value_counts().nlargest(5).tolist())
    return px.pie(filtered_df, names=filtered_df['breed'].value_counts().keys().tolist(), values=filtered_df['breed'].value_counts().tolist())


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(view_data):
    dff = pd.DataFrame.from_dict(view_data)
    markers = [dl.Marker(position=[row['location_lat'], row['location_long']], children=[
                dl.Tooltip(row['animal_type']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(row['animal_type'])
                ])
            ]) for index, row in dff.iterrows()]
    markers.insert(0, dl.TileLayer(id="base-layer-id"))
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=markers)
    ]


app